In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
from data_formatters.base import GenericDataFormatter, InputTypes, DataTypes
from data_formatters.sorgenia_wind import SorgeniaFormatter
from expt_settings.configs import ExperimentConfig
from libs.hyperparam_opt import HyperparamOptManager
from libs.tft_model import TemporalFusionTransformer
import libs.utils as utils
import os
import pandas as pd
from pandas import DataFrame, Series, Timestamp, Index
from tqdm import tqdm

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf1
from tensorflow.compat.v1 import Session, ConfigProto
from tensorflow.python.eager.context import PhysicalDevice
from typing import Dict, List, Union, Generator
from numpy import load

IMPORT FUNCTIONS TO TRANSFORM DF AND CALCULATE MAPE

In [19]:
from inference import mape
from inference import utils

In [4]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device:/device:GPU:0


In [5]:
gpu: List[PhysicalDevice] = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

In [6]:
# Tensorflow setup
default_keras_session: Session = tf1.keras.backend.get_session()
tf_config: ConfigProto = utils.get_default_tensorflow_config(tf_device="gpu", gpu_id=0)

Selecting GPU ID=0


In [7]:
file_path: str = r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs\data\sorgenia_wind\data\sorgenia_wind\data\sorgenia_final.csv'

In [8]:
# extract data into DataFrame
raw_data: DataFrame = pd.read_csv(file_path)
raw_data['time'] = raw_data['time'].astype('datetime64[s]') 

In [9]:
raw_data.head()

,plant_name_up,time,kwh,dew_point_2m_C,temperature_2m_C,msl_pressure_hPa,sfc_pressure_hPa,precipitation_1h_mm,wind_speed_mean_10m_1h_ms,wind_speed_mean_100m_1h_ms,...,days_from_start,id,hour,day,day_of_week,month,categorical_id,hours_from_start,categorical_day_of_week,categorical_hour
0,UP_MPNTLCDMRN_1,2019-01-01 00:00:00,3976.3125,2.8,3.4,1022.8,958.1,0.89,2.8,5.3,...,0,UP_MPNTLCDMRN_1,0,1,1,1,UP_MPNTLCDMRN_1,0.0,1,0
1,UP_MPNTLCDMRN_1,2019-01-01 01:00:00,3576.1875,0.1,4.6,1022.4,958.0,0.30,4.0,7.3,...,0,UP_MPNTLCDMRN_1,1,1,1,1,UP_MPNTLCDMRN_1,1.0,1,1
2,UP_MPNTLCDMRN_1,2019-01-01 02:00:00,3169.8750,-1.0,4.5,1021.8,957.6,0.07,4.7,8.6,...,0,UP_MPNTLCDMRN_1,2,1,1,1,UP_MPNTLCDMRN_1,2.0,1,2
3,UP_MPNTLCDMRN_1,2019-01-01 03:00:00,4007.6250,-1.0,4.8,1021.4,957.2,0.09,4.8,9.0,...,0,UP_MPNTLCDMRN_1,3,1,1,1,UP_MPNTLCDMRN_1,3.0,1,3
4,UP_MPNTLCDMRN_1,2019-01-01 04:00:00,4361.8125,0.6,4.4,1021.2,956.9,0.14,4.7,8.8,...,0,UP_MPNTLCDMRN_1,4,1,1,1,UP_MPNTLCDMRN_1,4.0,1,4


In [10]:
config = ExperimentConfig('sorgenia_wind', r'C:\Users\Lorenzo\PycharmProjects\TFT\outputs')

# START WITH MODEL1

Wind Forecasts as known input

In [11]:
formatter: SorgeniaFormatter = config.make_data_formatter()

In [12]:
data_csv_path: str = config.data_csv_path

# SPLIT DATA

In [13]:
train, valid, test = formatter.split_data(raw_data)

Setting scalers with training data...


# OPEN PREDICTIONS AND TARGETS FILES

In [42]:
def evaluate(predictions_path: str, model: str):
    p50_forecast: DataFrame = pd.read_csv(os.path.join(predictions_path, "p50.csv"))
    p90_forecast: DataFrame = pd.read_csv(os.path.join(predictions_path, "p90.csv"))
    targets: DataFrame = pd.read_csv(os.path.join(predictions_path, "targets.csv"))
    targets['forecast_time'] = targets['forecast_time'].astype('datetime64[s]')
    p50_forecast['forecast_time'] = p50_forecast['forecast_time'].astype('datetime64[s]')
    p90_forecast['forecast_time'] = p90_forecast['forecast_time'].astype('datetime64[s]')
    # pivot targets
    targets_piv: DataFrame = utils.pivot(targets)
    p50_piv: DataFrame = utils.pivot(p50_forecast)
    p90_piv: DataFrame = utils.pivot(p90_forecast)
    # check if df have same date range
    assert np.array_equal(targets_piv['forecast_time'].values, p50_piv['forecast_time'].values)
    assert np.array_equal(targets_piv['forecast_time'].values, p90_piv['forecast_time'].values)
    # calculate rolling mape on p50
    df_mape_50: DataFrame = mape.rolling_mape_multitarget(targets_piv, p50_piv, 700)
    print(f"{model} ", df_mape_50.iloc[:, 1:].mean().mean())

In [43]:
wind_known_path: str = r'C:\Users\Lorenzo\savedmodels_sorgenia_wind_unknown'

In [44]:
evaluate(wind_known_path, 'no_forecasts_model')

..\inference\utils.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df_piv2['level_1'] = df_piv2['level_1'].str.replace('t+', '')
..\inference\utils.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df_piv2['level_1'] = df_piv2['level_1'].str.replace('t+', '')
..\inference\utils.py:46: FutureWarning: The default value of regex will change from True to False in a future version.
  df_piv2['level_1'] = df_piv2['level_1'].str.replace('t+', '')


no_forecasts_model  64.73959397056852
